In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **1. Installation**

Load your dataset

In [ ]:
%tensorflow_version 1.x
!pip install --upgrade h5py==2.10.0
!git clone https://github.com/pysource7/Mask_RCNN
import sys
sys.path.append("/content/Mask_RCNN/demo")
from train_mask_rcnn_pro import *
%matplotlib inline

## **2. Image Dataset**

Load your annotated dataset


In [ ]:
# Extract Images
images_path = "/content/drive/MyDrive/data.zip"
annotations_path = "/content/annotations.json"

extract_images(os.path.join("/content/",images_path), "/content/dataset")

Extracted: 102 images


In [ ]:
dataset_train = load_image_dataset(os.path.join("/content/", annotations_path), "/content/dataset", "train")
dataset_val = load_image_dataset(os.path.join("/content/", annotations_path), "/content/dataset", "val")

class_number = dataset_train.count_classes()

print('Train: %d' % len(dataset_train.image_ids))
print('Validation: %d' % len(dataset_val.image_ids))
print("Classes: {}".format(class_number))

In [ ]:
# Load image samples
display_image_samples(dataset_train)

##**3. Training**

Train Mask RCNN on your custom Dataset.

In [ ]:
# Load Configuration
config = CustomConfig(class_number)
#config.display()
model = load_training_model(config,_)

In [ ]:
# Start Training
# This operation might take a long time.
train_head(model, dataset_train, dataset_train, config,5)

## **4. Detection (test your model on a random image)**

In [ ]:
# Load Test Model
# The latest trained model will be loaded
test_model, inference_config = load_test_model(class_number,_)

In [ ]:
test_random_image(test_model, dataset_val, inference_config)

In [ ]:
!python /content/Mask_RCNN/video_demo.py /content/drive/MyDrive/f1.mp4

In [ ]:
import cv2
import time
capture = cv2.VideoCapture("/content/drive/MyDrive/f1.mp4")
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size = (height, width)
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
# output_movie = cv2.VideoWriter('Zoo-video-masked.avi', fourcc, 30.0, (1920,1080))
video = cv2.VideoWriter("Zoo_video_masked.avi", cv2.VideoWriter_fourcc(*'DIVX'),fps, (1152, 1152))
# Constructing code of the codec to be used in the function VideoWriter
# writer = cv2.VideoWriter('Zoo-video-masked.mp4', fourcc, 20, (1920, 1080))
count=0
while True:
    # Capture frame-by-frame
    ret, frame = capture.read()
    if not ret:
        break

    results = model.detect([frame], verbose=1)
    r = results[0]

    boxes = r['rois']
    masks = r['masks']
    class_ids = r['class_ids']
    scores = r['scores']

    # Run detection
    start = time.time()
    masked_image = vc.display_instances(frame, boxes, masks, class_ids, class_names, scores)
    end = time.time()
    print("Inference time: {:.2f}s".format(end - start))

    # Display the resulting frame
    # cv2.imshow('', masked_image)
    #save image
    # writer = cv2.VideoWriter('Zoo-video-masked.mp4', fourcc, 30, (frame.shape[1], frame.shape[0]), True)
    #we are saving the image as "Masked-image.jpg under visualizeCustom code"
    #this image file we are writing inour videowriter object
    print("here,",type(masked_image))
    img=cv2.imread("Masked-image.jpg")
    print(img.shape)
    video.write(img)
    # output_movie.write(masked_image)
    count+=1
    print("count:",count)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

# When everything done, release the capture
print("WARNING")
cv2.destroyAllWindows()
capture.release()
video.release()
# cv2.destroyAllWindows()

In [ ]:

# example of inference with a pre-trained coco model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from mrcnn.visualize import display_instances
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
 
# define 81 classes that the coco model knowns about
class_names = ["ferrari","mclaren,","mercedes","rebull"]
 
# define the test configuration
class TestConfig(Config):
     NAME = "test"
     GPU_COUNT = 1
     IMAGES_PER_GPU = 1
     NUM_CLASSES = 1 + 4
 
# define the model
rcnn = MaskRCNN(mode='inference', model_dir='./', config=TestConfig())
# load coco model weights
rcnn.load_weights('/content/gdrive/MyDrive/pysource_mrcnn_pro/object20211027T2145/mask_rcnn_object_0005.h5', by_name=True)
# load photograph
img = load_img('/content/indir.jpg')
img = img_to_array(img)
# make prediction
results = rcnn.detect([img], verbose=0)
# get dictionary for first prediction
r = results[0]
# show photo with bounding boxes, masks, class labels and scores
display_instances(img, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])